In [21]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import roc_curve, auc, classification_report
import matplotlib.pyplot as plt

In [22]:
# Load the Breast Cancer Wisconsin dataset
url = '../data/wdbc.data'  # Path to your data file
columns = ['ID', 'diagnosis'] + [f'feature_{i}' for i in range(1, 31)]
data = pd.read_csv(url, header=None, names=columns)

In [23]:
# Preprocess the data: separate features and labels
X = data.drop(columns=["ID", "diagnosis"])  # Features
y = data["diagnosis"].map({'M': 1, 'B': 0})  # Labels: 1 for malignant, 0 for benign

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  

In [24]:
# Separate the benign (malignant) examples for training the positive (negative) autoencoder
X_benign = X_scaled[y == 0]  # Benign examples
X_malignant = X_scaled[y == 1]  # Malignant examples

# Split benign data into train and test sets
X_train_benign, X_test_benign = train_test_split(X_benign, test_size=0.2, random_state=42)

# Split malignant data into train and test sets
X_train_malignant, X_test_malignant = train_test_split(X_malignant, test_size=0.2, random_state=42)

# combine the test sets
X_test = np.concatenate((X_test_benign, X_test_malignant), axis=0)
y_test = np.concatenate((np.zeros(len(X_test_benign)), np.ones(len(X_test_malignant))), axis=0)


In [25]:
# Define and train the positive autoencoder on benign data
autoencoder_pos = MLPRegressor(hidden_layer_sizes=(14, 7, 14), activation='relu', max_iter=200, random_state=42)
autoencoder_pos.fit(X_train_benign, X_train_benign)

# Define and train the negative autoencoder on malignant data
autoencoder_neg = MLPRegressor(hidden_layer_sizes=(14, 7, 14), activation='relu', max_iter=200, random_state=42)
autoencoder_neg.fit(X_train_malignant, X_train_malignant)

/Users/bono/opt/miniconda3/envs/minenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/bono/opt/miniconda3/envs/minenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(14, 7, 14), random_state=42)

In [26]:
#get feature-level reconstruction errors for positive and negative autoencoder
X_test_pos = autoencoder_pos.predict(X_test)
X_test_neg = autoencoder_neg.predict(X_test)


# get the patient-level (ie point-level) reconstruction errors (ie the scores)
scores_pos = np.mean(np.power(X_test - X_test_pos, 2), axis=1)
scores_neg = np.mean(np.power(X_test - X_test_neg, 2), axis=1)


#combine the positive and negative scores
scores_dual = (scores_pos - scores_neg) / (scores_pos + scores_neg + 1e-8)  


In [27]:
# Calculate the ROC curve and AUC for the positive model
fpr_pos, tpr_pos, _ = roc_curve(y_test, scores_pos)
roc_auc_pos = auc(fpr_pos, tpr_pos)

# Calculate the ROC curve and AUC for the negative model
fpr_neg, tpr_neg, _ = roc_curve(y_test, scores_neg)
roc_auc_neg = auc(fpr_neg, tpr_neg)

In [28]:
# Calculate the ROC curve and AUC for positive model
fpr_pos, tpr_pos, thresholds = roc_curve(labels_pos, errors_pos)
roc_auc_pos = auc(fpr_pos, tpr_pos)

# Calculate the ROC curve and AUC for negative model
fpr_neg, tpr_neg, thresholds = roc_curve(labels_neg, errors_neg)
roc_auc_neg = auc(fpr_neg, tpr_neg)

# Plot the ROC curve
plt.figure()
#add the positive roc curve
plt.plot(fpr_pos, tpr_pos, color='darkorange', lw=2, label=f'ROC curve pos model (AUC = {roc_auc_pos:.2f})')
#add the negative roc curve 
plt.plot(fpr_neg, tpr_neg, color='red', lw=2, label=f'ROC curve neg model (AUC = {roc_auc_neg:.2f})')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Autoencoder Trained on Benign Examples')
plt.legend(loc='lower right')
plt.show()

# Print the AUC score
roc_auc_pos

NameError: name 'labels_pos' is not defined